### INTRODUCTION - This is the most basic model built for MNIST image classification with neural networks. Neural networks are an interconnected collection of nodes called neurons or perceptrons. Every neuron takes one piece of the input data, typically one pixel of the image, and applies a simple computation in the form of an activation function to generate a result.This example is set to explain basic concepts of data loading, neural network model building, metric evaluation and prediction. It calls for numpy random seed for reproducibilty purpose. It also explains basics of data engineering techniques such as normalization, data reshape and type conversion. It is a sequential model with single layer and kernel initialized to zero(The term kernel_initializer is a term for which statistical distribution or function to use for initialising the weights). Let's dive deep into the concepts.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

### np.random.seed(0) makes the random numbers predictable
#### >>> numpy.random.seed(0) ; numpy.random.rand(4)
#### array([ 0.55,  0.72,  0.6 ,  0.54])
#### >>> numpy.random.seed(0) ; numpy.random.rand(4)
#### array([ 0.55,  0.72,  0.6 ,  0.54])
#### With the seed reset (every time), the same set of numbers will appear every time.
#### If the random seed is not reset, different numbers appear with every invocation:
#### >>> numpy.random.rand(4)
#### array([ 0.42,  0.65,  0.44,  0.89])
#### >>> numpy.random.rand(4)
#### array([ 0.96,  0.38,  0.79,  0.53])
#### (pseudo-)random numbers work by starting with a number (the seed), multiplying it by a large number, adding an offset, then taking modulo of that sum. The resulting number is then used as the seed to generate the next "random" number. When you set the seed (every time), it does the same thing every time, giving you the same numbers.
#### If you want seemingly random numbers, do not set the seed. If you have code that uses random numbers that you want to debug, however, it can be very helpful to set the seed before each run so that the code does the same thing every time you run it.
#### To get the most random numbers for each run, call numpy.random.seed(). This will cause numpy to set the seed to a random number obtained from /dev/urandom or its Windows analog or, if neither of those is available, it will use the clock.
#### Reference - https://stackoverflow.com/questions/21494489/what-does-numpy-random-seed0-do


In [ ]:
# for reproducibility
np.random.seed(1671)

### In neural networks, one epoch means one forward pass and one backward pass of all the training examples. Example: if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete 1 epoch.
### Reference - https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network

In [ ]:
EPOCHS = 200

### Batch size stands for the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need. Batch size defines the number of samples that will be propagated through the network.
#### For instance, let's say you have 1050 training samples and you want to set up a batch_size equal to 100. The algorithm takes the first 100 samples (from 1st to 100th) from the training dataset and trains the network. Next, it takes the second 100 samples (from 101st to 200th) and trains the network again. We can keep doing this procedure until we have propagated all samples through of the network. 
#### Advantages of using a batch size < number of all samples:
#### •	It requires less memory. Since you train the network using fewer samples, the overall training procedure requires less memory. That's especially important if you are not able to fit the whole dataset in your machine's memory.
#### •	Typically networks train faster with mini-batches. That's because we update the weights after each propagation. In our example we've propagated 11 batches (10 of them had 100 samples and 1 had 50 samples) and after each of them we've updated our network's parameters. 

#### Reference - https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network

In [ ]:
BATCH_SIZE = 128

### By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
#### verbose=0 will show you nothing (silent)
#### verbose=1 will show you an animated progress bar like this:
#### verbose=2 will just mention the number of epoch like this:
#### Reference - https://stackoverflow.com/questions/47902295/what-is-the-use-of-verbose-in-keras-while-validating-the-model

In [ ]:
VERBOSE = 1

### It is the number of outputs

In [ ]:
NB_CLASSES = 10   # number of outputs = number of digits

### Positive integer, dimensionality of the output space that will be produced.

In [ ]:
N_HIDDEN = 128

### It is the amount of data reserved for checking or proving the validity of the training process.

In [ ]:
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

In [ ]:
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


### Here, the training and testing data is being normalized by scaling it within a range of 0 and 1 as every value lies between 0-255.
https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/

In [ ]:
#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0

### Reshape value for 28*28 matrix

In [ ]:
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784

### This is reshaping the data from one form to the other. Here X_train is 60000 rows of 28x28 values. Therefore, we reshape it to 60000 x 784. Reshaping is done because keras convolution layers work with higher dimensions.

https://stackoverflow.com/questions/52089601/keras-dense-layers-input-is-not-flattened/52092176#52092176

In [ ]:
X_train = X_train.reshape(60000, RESHAPED)

### This is reshaping the data from one form to the other. Here X_test is 10000 rows of 28x28 values. Therefore, we reshape it to 10000 x 784. Reshaping is done because keras convolution layers work with higher dimensions.

In [ ]:
X_test = X_test.reshape(10000, RESHAPED)

### This converts the type to 32 bit float.

In [ ]:
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')

### Sequential groups a linear stack of layers into a tf.keras.Model. Sequential provides training and inference features on this model.A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
#### A Sequential model is not appropriate when:
#### •	Your model has multiple inputs or multiple outputs
#### •	Any of your layers has multiple inputs or multiple outputs
#### •	You need to do layer sharing
#### •	You want non-linear topology (e.g. a residual connection, a multi-branch model)
#### Reference - https://keras.io/guides/sequential_model/

In [ ]:
model = tf.keras.models.Sequential()

Each neuron can be initialized with specific weights via the kernel_initializer parameter. There are a few choices, the most common of which are listed as follows:

• random_uniform: Weights are initialized to uniformly random small values in the range -0.05 to 0.05.

• random_normal: Weights are initialized according to a Gaussian distribution, with zero mean and a small standard deviation of 0.05. For those of you who are not familiar with Gaussian distribution, think about a symmetric "bell curve" shape.

• zero: All weights are initialized to zero.

A full list is available online at https://www.tensorflow.org/api_docs/python/
tf/keras/initializers.

### The final layer is a single neuron with activation function "softmax", which is a generalization of the sigmoid function. A sigmoid function output is in the range (0, 1) when the input varies in the range (−∞, ∞). Similarly, a softmax "squashes" a K-dimensional vector of arbitrary real values into a K-dimensional vector of real values in the range (0, 1), so that they all add up to 1. In our case, it aggregates 10 answers provided by the previous layer with 10 neurons.

In [ ]:
model.add(keras.layers.Dense(NB_CLASSES,
       input_shape=(RESHAPED,), kernel_initializer='zeros',
       name='dense_layer', activation='softmax'))


In [ ]:
# summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


### model.compile() method compiles and creates a neural network model. It takes the following parameters – 
### There are a few choices to be made during compilation. Firstly, we need to select an optimizer, which is the specific algorithm used to update weights while we train our model. Second, we need to select an objective function, which is used by the optimizer to navigate the space of weights (frequently, objective functions are called either loss functions or cost functions. Third, we need  to evaluate the trained model.
### optimizer - Optimizers are algorithms or methods used to change the attributes of your neural network such as weights and learning rate in order to reduce the losses.
### Various optimizers - https://medium.com/@sdoshi579/optimizers-for-training-neural-network-59450d71caf6
### Loss - The Loss Function is one of the important components of Neural Networks. Loss is nothing but a prediction error of Neural Net. And the method to calculate the loss is called Loss Function. In simple words, the Loss is used to calculate the gradients.
#### Reference - https://towardsdatascience.com/understanding-different-loss-functions-for-neural-networks-dd1ed0274718
### Metrics - A metric is a function that is used to judge the performance of your model. Metric functions are similar to loss functions, except that the results from evaluating a metric are not used when training the model. Note that you may use any loss functions as a metric function.
#### Different metrics and reference - https://keras.io/api/metrics/

In [ ]:
# compiling the model
model.compile(optimizer='SGD', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#training the moodel
model.fit(X_train, Y_train,
        batch_size=BATCH_SIZE, epochs=EPOCHS,
        verbose=VERBOSE, validation_split=VALIDATION_SPLIT)


Epoch 1/200
375/375 [==============================] - 1s 3ms/step - loss: 1.3527 - accuracy: 0.7811 - val_loss: 0.8783 - val_accuracy: 0.8418
Epoch 2/200
375/375 [==============================] - 1s 3ms/step - loss: 0.7803 - accuracy: 0.8417 - val_loss: 0.6478 - val_accuracy: 0.8642
Epoch 3/200
375/375 [==============================] - 1s 3ms/step - loss: 0.6349 - accuracy: 0.8569 - val_loss: 0.5548 - val_accuracy: 0.8754
Epoch 4/200
375/375 [==============================] - 1s 3ms/step - loss: 0.5645 - accuracy: 0.8661 - val_loss: 0.5033 - val_accuracy: 0.8812
Epoch 5/200
375/375 [==============================] - 1s 3ms/step - loss: 0.5215 - accuracy: 0.8722 - val_loss: 0.4700 - val_accuracy: 0.8867
Epoch 6/200
375/375 [==============================] - 1s 3ms/step - loss: 0.4921 - accuracy: 0.8761 - val_loss: 0.4465 - val_accuracy: 0.8888
Epoch 7/200
375/375 [==============================] - 1s 3ms/step - loss: 0.4702 - accuracy: 0.8800 - val_loss: 0.4291 - val_accuracy: 0.8910

In [ ]:
#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.2770 - accuracy: 0.9212
Test accuracy: 0.9211999773979187


In [ ]:
# making prediction
predictions = model.predict(X_test)

**Observations**

We have acheived the training accuracy of 92.9,validation of 9237 and test accuracy of 92.1%